# Introduction

In this notebook, I will demonstrate how to embed D3.js visualizations within IPython notebooks. It has been  acknowledged that visualizing D3.js plots has proven to be rather technically challenging on Kaggle notebooks and the only one that I have come across that has managed to do integrate D3.js with kernels on a consistent and a quality basis is [Triskelion](https://www.kaggle.com/triskelion). (Do check out his work as they are of top-notch quality and incidentally, he was the Kaggler that really got me intrigued with Kernels ). However, with the revamping of kernel interface and the backend, unfortunately it seems that Triskelion's old D3.js kaggle scripts are now not able to be run the way they were before before. Therefore I am hoping that this notebook will provide a new way of visualising D3.js plots and become of use to those visualization enthusiasts out there.

**Brief history on D3.js** 

[D3.js (a.k.a Data Driven Documents) ](https://d3js.org/)  is a javascript library which was created with the intent on serving beautiful, interactive and dynamic visualizations on web browsers and was developed by the team of Jeff Heer, Mike Bostock, and Vadim Ogievetsky of Stanford University's Stanford Visualization Group. The drawing power of this library is that unlike other visualization tools where one is constrained in the type of plot that she/he can call (e.g. bar charts, line, radial plots etc), D3 allows a user to customize the final visuals to a very fine degree of granularity, whilst keeping in line with the web standards of HTML5, CSS and SVG. 

To quote Wikipedia's page: 

*the D3.js library uses pre-built JavaScript functions to select elements, create SVG objects, style them, or add transitions, dynamic effects or tooltips to them. These objects can also be widely styled using CSS. Large datasets can be easily bound to SVG objects using simple D3.js functions to generate rich text/graphic charts and diagrams.*

The type of files that D3.js can take in are usually of the format .json (Javascript object notation) as well as .csv (comma separated values). There are other types of files that are accepted, such as geoJson but generally the most common types that one would come across are the two aforementioned formats. This notebook is arranged according to these chapters:

1. [Investigating the Class feature for groupings to add to the circle-packing](#circle_pack)
2. [Data Preparation of the input JSON file](#json)
3. [Plotting D3 visuals via embedding Javascript](#plot)

Let's go

In [ ]:
from IPython.core.display import display, HTML, Javascript
from string import Template
import pandas as pd
import numpy as np
import json, random
import IPython.display

### What is Circle-packing?

The particular type of visualisation I want to showcase is known as circle-packing, which is the arrangement of circles inside some demarcation so that none of the circles overlap. In fact, the mathematics of circle-packing can indeed get quite extensive, but from a visual standpoint it allows one to easily and intuitively observe clusters of similar concepts based on how tightly the circles are packed. Circle packing also displays hierarchy where you can get smaller clusters of circles packed within a bigger circle which itself is arranged next to or within other circles- given its hierarchical nature, it is no surprise that circle packing is also sometimes known as the circular treemap. 

The picture show below is a taste of what we will be going to create in the latter sections of this notebook. However unlike this static figure, the D3.js plot will be interactive and dynamic, where one is able to invoke zoomable animations at different regions and clusters with the click of a mouse button. Note the one drawback of the circle packing method is that it is not very space efficient as compared to a standard rectangular as there is a lot of unused space in the visuals. However the free space also has its pros as it does not clutter up the entire view when one looks at the plot which make actually make for more appealing visuals. As a base script, I will use Mike Bostocks excellent example : [Zoomable Circle packing](https://bl.ocks.org/mbostock/7607535).

Sounds interesting? If so let us proceed.

In [ ]:
import matplotlib.pyplot as plt
# from scipy.misc import imread
from imageio import imread
import base64
import codecs
import cv2
circle_pack = b'...'
f1 = open("circle_pack.png", "wb")
f1.write(codecs.decode(circle_pack,'base64'))
f1.close()
cp = imread("circle_pack.png")
# sing = cv2.cvtColor(cp)
plt.figure(figsize = (15,12))
plt.imshow(cp)

Load in the training data as follows. Note that there are not many columns in our dataset. 

In [ ]:
train = pd.read_csv('../input/en_train.csv')
train.head(5)

Nonetheless, the class feature seems like a most interesting column to investigate and plot, especially given the context of this kernel which is to showcase the circle-packing visuals. By calling the pandas dataframe method value_counts(), we now have a clearer picture of 

In [ ]:
train['class'].value_counts()

<a id='circle_pack'></a>
# Circle-packing the Class feature

The plan therefore is to simply look at some of the categories within the class feature of the data, dive deeper within each category and look at what are the values associated to each category and then plot clusters of the values and class categories within a circle packed treemap. The visualisation should therefore help us to intutitvely identify clusters by size as well as their associated values. 

In [ ]:
plain = train[train['class']=='PLAIN'].iloc[:75,:]
punctuation = train[train['class']=='PUNCT'].iloc[:18,:]
date = train[train['class']=='DATE'].iloc[:15,:]
letters = train[train['class']=='LETTERS'].iloc[:10,:]
cardinal = train[train['class']=='CARDINAL'].iloc[:10,:]
verbatim = train[train['class']=='VERBATIM'].iloc[:5,:]
measure = train[train['class']=='MEASURE'].iloc[:35,:]        

In [ ]:
vcounts = train['class'].value_counts()

----
<a id="json"></a>
# Data preparation of Json file

To start off, we  first concern ourselves with preparing the data to be fed into D3. In this instance, I will prepare a Python dictionary which will ultimately be converted into a .json file to be read back by the javascript for plotting. Please unhide the code below to see the full structure of the dictionary.

For the purposes of this kernel in showcasing the visualisation capabilties of D3.js, I will simply identify groupings within the different classes and manually feed that into the json object based off the values obtained from the variables plain, punctuation, date et al - NOTE this is not good programming and only used here for brevity and convenience. One should really write code to dynamically create the json.
Also the groupings do not explicitly exist so you are free to use your creativity to model any groupings you may want to particularly display or potentially use some sort of latent or clustering model to learn the groupings from the data. For example in the DATE class there are many different years provided, but I want to visually identify all the dates after year 2000 as a child circle within the Date circle and therefore I store all dates after year 2000 as a sub-dictionary (code below).

In a commercial or business setting, you would ultimately want to implement some programmatic process that models your json file based on the data - e.g. finding groupings or clusters within a class text using some clustering method and feeding those clusters as sub-circles within the outer parent json nodes.

In [ ]:
flare_json = {
 "name": "flare",
 "children": [
  {
   "name": "CARDINAL",
   "children": [
    {
     "name": "Single_digit",
     "children": [
      {"name": "4", "size": 3938},
      {"name": "2", "size": 3812},
      {"name": "3", "size": 6714},
      {"name": "9", "size": 1812}
     ]
    },
    {
     "name": "double_triple",
     "children": [
      {"name": "35", "size": 3534},
      {"name": "91", "size": 5731},
      {"name": "935", "size": 7840}
     ]
    },
    {
     "name": "Many_digits",
     "children": [
      {"name": "1065520", "size": 7074}
     ]
    }
   ]
  },
  {
   "name": "DATE",
   "children": [
    {"name": "Sep 2015", "size": 17010},
    {"name": "Dec 2015", "size": 5842},
    {
     "name": "Before Year 2000",
     "children": [
      {"name": "1987", "size": 1983},
      {"name": "1996", "size": 2047},
      {"name": "1982", "size": 1375},
      {"name": "1916", "size": 8746},
      {"name": "1989", "size": 2202}
     ]
    },
    {"name": "2006", "size": 1041},
    {"name": "2015", "size": 5176},
    {"name": "2008", "size": 449},
    {"name": "2014", "size": 5593},
    {"name": "2013", "size": 5534},
    {"name": "2007", "size": 9201},
    {"name": "2014", "size": 19975},
    {"name": "2005", "size": 6006}
   ]
  },
  {
   "name": "LETTERS",
   "children": [
    {"name": "IUCN", "size": 1759},
    {"name": "BC", "size": 2165},
    {"name": "ALCS", "size": 586},
    {"name": "Aceh", "size": 3331},
    {"name": "C.", "size": 772},
    {"name": "G.", "size": 772},
    {"name": "U.S.", "size": 772},
    {"name": "LP", "size": 772},
    {"name": "J.", "size": 3322}
   ]
  },
  {
   "name": "VERBATIM",
   "children": [
    {"name": "-", "size": 8833},
    {"name": "#", "size": 1732},
    {"name": '&', "size": 3623},
    {"name": "イ", "size": 4833},
    {"name": 'サ', "size": 10066}
   ]
  },
  {
   "name": "MEASURE",
   "children": [
    {"name": "100 m", "size": 1616},
    {"name": "60km", "size": 1027},
    {"name": "1GB", "size": 3891},
    {"name": "578.3/km2", "size": 891},
    {"name": "43 km", "size": 2893},
    {"name": "26.7 mi", "size": 5103},
    {"name": "459.0/km²", "size": 3677},
    {"name": "212.6/km²", "size": 781},
    {"name": "1,000ft", "size": 4141},
    {"name": "13 kg", "size": 933},
    {"name": "29 lb", "size": 5130},
    {"name": "37.0/km²", "size": 3617},
    {"name": "3.04 km2", "size": 3240},
    {"name": "1.17 sq mi", "size": 2732},
    {"name": "218.204 mph", "size": 2039},
    {"name": "50 nm", "size": 1214},
    {"name": "208.913 mph", "size": 3748},
    {"name": "4000ha", "size": 843},
    {
     "name": "Percentages",
     "children": [
      {"name": "60%", "size": 593},
      {"name": "16.4%", "size": 330},
      {"name": "57.6%", "size": 287},
      {"name": "79.20%", "size": 277},
      {"name": "59.30%", "size": 292},
      {"name": "63%", "size": 595},
      {"name": "50%", "size": 594},
      {"name": "50%", "size": 460},
      {"name": "10%", "size": 603},
      {"name": "45%", "size": 625},
      {"name": "9.9%", "size": 748},
      {"name": "4.2%", "size": 461},
      {"name": "6%", "size": 597},
      {"name": "12%", "size": 619},
      {"name": "40%", "size": 283},
      {"name": "16.4%", "size": 283},
      {"name": "79.20%", "size": 591},
      {"name": "57.6%", "size": 603},
      {"name": "45%", "size": 599},
      {"name": "9.9%", "size": 386},
      {"name": "4.2%", "size": 323},
      {"name": "6%", "size": 307},
      {"name": "12%", "size": 772},
      {"name": "16.4%", "size": 296},
      {"name": "59%", "size": 363},
      {"name": "_", "size": 264}
     ]
    },
    {"name": "351.165 km/h", "size": 843},
    {"name": '40 km²', "size": 1554},
    {"name": '10.56 km²', "size": 970},
    {"name": '336.213 km/h', "size": 13896},
    {"name": '1.17 sq mi', "size": 1594},
    {"name": '3.04 km2', "size": 4130},
    {"name": '37.0/km²', "size": 791},
    {"name": '1,000ft', "size": 1124},
    {"name": '29 lb', "size": 1876},
    {"name": '13 kg', "size": 1101}
   ]
  },
  {
   "name": "scale",
   "children": [
    {"name": "IScaleMap", "size": 2105},
    {"name": "LinearScale", "size": 1316},
    {"name": "LogScale", "size": 3151},
    {"name": "OrdinalScale", "size": 3770},
    {"name": "QuantileScale", "size": 2435},
    {"name": "QuantitativeScale", "size": 4839},
    {"name": "RootScale", "size": 1756},
    {"name": "Scale", "size": 4268},
    {"name": "ScaleType", "size": 1821},
    {"name": "TimeScale", "size": 5833}
   ]
  },
  {
   "name": "PUNCTUATION",
   "children": [
    {"name": "Arrays", "size": 8258},
    {"name": "Colors", "size": 10001},
    {"name": "Dates", "size": 8217},
    {"name": "Displays", "size": 12555},
    {"name": "Filter", "size": 2324},
    {"name": "Geometry", "size": 10993},
    {
     "name": "heap",
     "children": [
      {"name": "FibonacciHeap", "size": 9354},
      {"name": "HeapNode", "size": 1233}
     ]
    },
    {"name": "IEvaluable", "size": 335},
    {"name": "IPredicate", "size": 383},
    {"name": "IValueProxy", "size": 874},
    {
     "name": "math",
     "children": [
      {"name": "DenseMatrix", "size": 3165},
      {"name": "IMatrix", "size": 2815},
      {"name": "SparseMatrix", "size": 3366}
     ]
    },
    {"name": "Maths", "size": 17705},
    {"name": "Orientation", "size": 1486},
    {
     "name": "palette",
     "children": [
      {"name": "ColorPalette", "size": 6367},
      {"name": "Palette", "size": 1229},
      {"name": "ShapePalette", "size": 2059},
      {"name": "SizePalette", "size": 2291}
     ]
    },
    {"name": "Property", "size": 5559},
    {"name": "Shapes", "size": 19118},
    {"name": "Sort", "size": 6887},
    {"name": "Stats", "size": 6557},
    {"name": "Strings", "size": 22026}
   ]
  },
  {
   "name": "PLAIN",
   "children": [
    {
     "name": "axis",
     "children": [
      {"name": "Axes", "size": 1302},
      {"name": "Axis", "size": 24593},
      {"name": "AxisGridLine", "size": 652},
      {"name": "AxisLabel", "size": 636},
      {"name": "CartesianAxes", "size": 6703}
     ]
    },
    {
     "name": "controls",
     "children": [
      {"name": "AnchorControl", "size": 2138},
      {"name": "ClickControl", "size": 3824},
      {"name": "Control", "size": 1353},
      {"name": "ControlList", "size": 4665},
      {"name": "DragControl", "size": 2649},
      {"name": "ExpandControl", "size": 2832},
      {"name": "HoverControl", "size": 4896},
      {"name": "IControl", "size": 763},
      {"name": "PanZoomControl", "size": 5222},
      {"name": "SelectionControl", "size": 7862},
      {"name": "TooltipControl", "size": 8435}
     ]
    },
    {
     "name": "data",
     "children": [
      {"name": "Data", "size": 20544},
      {"name": "DataList", "size": 19788},
      {"name": "DataSprite", "size": 10349},
      {"name": "EdgeSprite", "size": 3301},
      {"name": "NodeSprite", "size": 19382},
      {
       "name": "render",
       "children": [
        {"name": "ArrowType", "size": 698},
        {"name": "EdgeRenderer", "size": 5569},
        {"name": "IRenderer", "size": 353},
        {"name": "ShapeRenderer", "size": 2247}
       ]
      },
      {"name": "ScaleBinding", "size": 11275},
      {"name": "Tree", "size": 7147},
      {"name": "TreeBuilder", "size": 9930}
     ]
    },
    {
     "name": "events",
     "children": [
      {"name": "DataEvent", "size": 2313},
      {"name": "SelectionEvent", "size": 1880},
      {"name": "TooltipEvent", "size": 1701},
      {"name": "VisualizationEvent", "size": 1117}
     ]
    },
    {
     "name": "legend",
     "children": [
      {"name": "Legend", "size": 20859},
      {"name": "LegendItem", "size": 4614},
      {"name": "LegendRange", "size": 10530}
     ]
    },
    {
     "name": "operator",
     "children": [
      {
       "name": "distortion",
       "children": [
        {"name": "BifocalDistortion", "size": 4461},
        {"name": "Distortion", "size": 6314},
        {"name": "FisheyeDistortion", "size": 3444}
       ]
      },
      {
       "name": "encoder",
       "children": [
        {"name": "ColorEncoder", "size": 3179},
        {"name": "Encoder", "size": 4060},
        {"name": "PropertyEncoder", "size": 4138},
        {"name": "ShapeEncoder", "size": 1690},
        {"name": "SizeEncoder", "size": 1830}
       ]
      },
      {
       "name": "filter",
       "children": [
        {"name": "FisheyeTreeFilter", "size": 5219},
        {"name": "GraphDistanceFilter", "size": 3165},
        {"name": "VisibilityFilter", "size": 3509}
       ]
      },
      {"name": "IOperator", "size": 1286},
      {
       "name": "label",
       "children": [
        {"name": "Labeler", "size": 9956},
        {"name": "RadialLabeler", "size": 3899},
        {"name": "StackedAreaLabeler", "size": 3202}
       ]
      },
      {
       "name": "layout",
       "children": [
        {"name": "AxisLayout", "size": 6725},
        {"name": "BundledEdgeRouter", "size": 3727},
        {"name": "CircleLayout", "size": 9317},
        {"name": "CirclePackingLayout", "size": 12003},
        {"name": "DendrogramLayout", "size": 4853},
        {"name": "ForceDirectedLayout", "size": 8411},
        {"name": "IcicleTreeLayout", "size": 4864},
        {"name": "IndentedTreeLayout", "size": 3174},
        {"name": "Layout", "size": 7881},
        {"name": "NodeLinkTreeLayout", "size": 12870},
        {"name": "PieLayout", "size": 2728},
        {"name": "RadialTreeLayout", "size": 12348},
        {"name": "RandomLayout", "size": 870},
        {"name": "StackedAreaLayout", "size": 9121},
        {"name": "TreeMapLayout", "size": 9191}
       ]
      },
      {"name": "Operator", "size": 2490},
      {"name": "OperatorList", "size": 5248},
      {"name": "OperatorSequence", "size": 4190},
      {"name": "OperatorSwitch", "size": 2581},
      {"name": "SortOperator", "size": 2023}
     ]
    },
    {"name": "Visualization", "size": 16540}
   ]
  }
 ]
}
          

Having created the structure that we want to visualise out data with, we can now proceed to modify the Python dictionary into a json file which we can then feed into D3 for plotting. 

To do this, we use the **json** module and by invoking the function "dump", we are able to conveniently send each element in the dictionary off into a json file of our choosing, here in this case "output.json". This json file will come in handy later where we will call this file from the embedded javascript in the latter section.

In [ ]:
with open('output.json', 'w') as outfile:  
    json.dump(flare_json, outfile)
pd.read_json('output.json').head(10)

----
<a id='plot'></a>
# Plotting the D3.js visuals

To plot the D3 visuals, we first need to feed into it the HTML structure that will be used to embed our D3 SVG (Scalable Visual Graphic) object. To embed HTML into Kaggle notebooks, I store the relevant HTML into a Python string within the Python triple quotation marks. 

In [ ]:
html_string = """
<!DOCTYPE html>
<meta charset="utf-8">
<style>

.node {
  cursor: pointer;
}

.node:hover {
  stroke: #000;
  stroke-width: 1.5px;
}

.node--leaf {
  fill: white;
}

.label {
  font: 11px "Helvetica Neue", Helvetica, Arial, sans-serif;
  text-anchor: middle;
  text-shadow: 0 1px 0 #fff, 1px 0 0 #fff, -1px 0 0 #fff, 0 -1px 0 #fff;
}

.label,
.node--root,
.node--leaf {
  pointer-events: none;
}

</style>
<svg width="760" height="760"></svg>
"""

### Javascript embedding

Next we need to feed the base javascript that will ultimately be used to power our D3 plots. I have hidden the code for neatness but feel free to unhide the cell to peruse the full thing. 

In [ ]:
js_string="""
 require.config({
    paths: {
        d3: "https://d3js.org/d3.v4.min"
     }
 });

  require(["d3"], function(d3) {

   console.log(d3);

var svg = d3.select("svg"),
    margin = 20,
    diameter = +svg.attr("width"),
    g = svg.append("g").attr("transform", "translate(" + diameter / 2 + "," + diameter / 2 + ")");

var color = d3.scaleSequential(d3.interpolateViridis)
    .domain([-4, 4]);

var pack = d3.pack()
    .size([diameter - margin, diameter - margin])
    .padding(2);

d3.json("output.json", function(error, root) {
  if (error) throw error;

  root = d3.hierarchy(root)
      .sum(function(d) { return d.size; })
      .sort(function(a, b) { return b.value - a.value; });

  var focus = root,
      nodes = pack(root).descendants(),
      view;

  var circle = g.selectAll("circle")
    .data(nodes)
    .enter().append("circle")
      .attr("class", function(d) { return d.parent ? d.children ? "node" : "node node--leaf" : "node node--root"; })
      .style("fill", function(d) { return d.children ? color(d.depth) : null; })
      .on("click", function(d) { if (focus !== d) zoom(d), d3.event.stopPropagation(); });

  var text = g.selectAll("text")
    .data(nodes)
    .enter().append("text")
      .attr("class", "label")
      .style("fill-opacity", function(d) { return d.parent === root ? 1 : 0; })
      .style("display", function(d) { return d.parent === root ? "inline" : "none"; })
      .text(function(d) { return d.data.name; });

  var node = g.selectAll("circle,text");

  svg
      .style("background", color(-1))
      .on("click", function() { zoom(root); });

  zoomTo([root.x, root.y, root.r * 2 + margin]);

  function zoom(d) {
    var focus0 = focus; focus = d;

    var transition = d3.transition()
        .duration(d3.event.altKey ? 7500 : 750)
        .tween("zoom", function(d) {
          var i = d3.interpolateZoom(view, [focus.x, focus.y, focus.r * 2 + margin]);
          return function(t) { zoomTo(i(t)); };
        });

    transition.selectAll("text")
      .filter(function(d) { return d.parent === focus || this.style.display === "inline"; })
        .style("fill-opacity", function(d) { return d.parent === focus ? 1 : 0; })
        .on("start", function(d) { if (d.parent === focus) this.style.display = "inline"; })
        .on("end", function(d) { if (d.parent !== focus) this.style.display = "none"; });
  }

  function zoomTo(v) {
    var k = diameter / v[2]; view = v;
    node.attr("transform", function(d) { return "translate(" + (d.x - v[0]) * k + "," + (d.y - v[1]) * k + ")"; });
    circle.attr("r", function(d) { return d.r * k; });
  }
});
  });
 """

Finally to display the html, as well as the underlying javascript which will ultimately power the D3 visuals, we invoke the following code:

In [ ]:
h = display(HTML(html_string))
j = IPython.display.Javascript(js_string)
IPython.display.display_javascript(j)

*PLEASE CLICK ON THE CLUSTERS ABOVE. THESE PLOTS ARE INTERACTIVE AND ZOOMABLE. CLICK ON THE OUTSIDE IF YOU WANT TO GET BACK TO THE ORIGINAL VIEW.*

## Conclusion

In this notebook, I hope that I have managed to convey the power of the javascript visualization framework D3 (Data driven documents). Here we have demonstrated how to embed html and javasript syntax into Ipython notebooks, obtained a brief overview of the framework as well as showed how to implement a particular kind of treemap by packing circles iteratively. 